# generate_error_table

## imports

In [2]:
import pandas as pd
import numpy as np

## load data

In [3]:
df0 = pd.read_csv('data/20230221/equaldistinctcount_20230221.csv')
df1 = pd.read_csv('data/20230221/equiheight_20230221.csv')
df2 = pd.read_csv('data/20230221/equiwidth_20230221.csv')
df3 = pd.read_csv('data/20230221/gdy_20230221.csv')
df4 = pd.read_csv('data/20230221/maxdiff_20230221.csv')
df = pd.concat([df0, df1, df2, df3, df4], axis=0)
#df = df.fillna('TPC-DS')

## generate errors

In [4]:
def generate_errors(df):
    df = df.copy()  # Convenience
    
    df['absolute_error'] = np.absolute(df['estimated_output'] - df['real_output'])
    df['relative_error'] = df['absolute_error'] / df['real_output']
    df['x'] = df['estimated_output'] / df['real_output']
    df['1/x'] = 1 / df['x']
    df['q_error'] = df[['x','1/x']].max(axis=1)
    df['real_output_lower_bound'] = df['real_output']
    df['real_output_lower_bound'] = df['real_output_lower_bound'].clip(1)
    df['estimated_output_lower_bound'] = df['estimated_output']
    df['estimated_output_lower_bound'] = df['estimated_output_lower_bound'].clip(1)
    df['pseudo_x'] = df['estimated_output_lower_bound'] / df['real_output_lower_bound']
    df['pseudo_1/x'] = 1 / df['pseudo_x']
    df['pseudo_q_error'] = df[['pseudo_x','pseudo_1/x']].max(axis=1)
    df['mean_squared_error'] = df['absolute_error'] ** 2
    df.drop(['x', '1/x', 'real_output_lower_bound', 'estimated_output_lower_bound', 'pseudo_x', 'pseudo_1/x'], axis=1, inplace = True)

    return df

## result

### with_NULLs

In [5]:
df_with_errors = generate_errors(df)
df_with_errors

df_with_errors.groupby([#"operator_type",
                        "benchmark",
                        "histogram"]).agg(root_mean_squared_error=("mean_squared_error", lambda x: np.sqrt(np.mean(x))),
                                          mean_absolute_error=("absolute_error", np.mean),
                                          mean_relative_error=("relative_error", np.mean),
                                          mean_q_error=("q_error", np.mean),
                                          mean_pseudo_q_error=("pseudo_q_error", np.mean))

root_mean_squared_error  \
benchmark histogram                                              
JoinOrder EqualDistinctCountHistogram             4.436989e+05   
          EquiHeightHistogram                     4.923148e+05   
          EquiWidthHistogram                      4.961650e+05   
          GDYHistogram                            4.950823e+05   
          MaxDiffFrequencyHistogram               5.053823e+05   
TPC-DS    EqualDistinctCountHistogram             1.412218e+06   
          EquiHeightHistogram                     1.414673e+06   
          EquiWidthHistogram                      1.475394e+06   
          GDYHistogram                            1.405704e+06   
          MaxDiffFrequencyHistogram               1.409846e+06   
TPC-H     EqualDistinctCountHistogram             8.342819e+06   
          EquiHeightHistogram                     8.619737e+06   
          EquiWidthHistogram                      8.635061e+06   
          GDYHistogram                            8.342298e+06   
          MaxDiffFrequencyHistogram               8.346457e+06   

                                       mean_absolute_error  \
benchmark histogram                                          
JoinOrder EqualDistinctCountHistogram         1.162157e+05   
          EquiHeightHistogram                 1.180125e+05   
          EquiWidthHistogram                  1.180460e+05   
          GDYHistogram                        1.173108e+05   
          MaxDiffFrequencyHistogram           1.241232e+05   
TPC-DS    EqualDistinctCountHistogram         6.265854e+05   
          EquiHeightHistogram                 6.302854e+05   
          EquiWidthHistogram                  6.536177e+05   
          GDYHistogram                        5.858548e+05   
          MaxDiffFrequencyHistogram           6.169429e+05   
TPC-H     EqualDistinctCountHistogram         1.854357e+06   
          EquiHeightHistogram                 2.081983e+06   
          EquiWidthHistogram                  2.141277e+06   
          GDYHistogram                        1.863193e+06   
          MaxDiffFrequencyHistogram           1.872331e+06   

                                       mean_relative_error  mean_q_error  \
benchmark histogram                                                        
JoinOrder EqualDistinctCountHistogram                  NaN           NaN   
          EquiHeightHistogram                          NaN           NaN   
          EquiWidthHistogram                           NaN           NaN   
          GDYHistogram                                 NaN           NaN   
          MaxDiffFrequencyHistogram                    NaN           NaN   
TPC-DS    EqualDistinctCountHistogram                  NaN           NaN   
          EquiHeightHistogram                          NaN           NaN   
          EquiWidthHistogram                           NaN           NaN   
          GDYHistogram                           44.320481           NaN   
          MaxDiffFrequencyHistogram                    NaN           NaN   
TPC-H     EqualDistinctCountHistogram         26100.439606  26114.213733   
          EquiHeightHistogram                 30284.239488  30298.063391   
          EquiWidthHistogram                  31512.937851  31519.749527   
          GDYHistogram                        30628.350620           NaN   
          MaxDiffFrequencyHistogram           32614.381777  32780.924211   

                                       mean_pseudo_q_error  
benchmark histogram                                         
JoinOrder EqualDistinctCountHistogram         26587.658467  
          EquiHeightHistogram                 11669.086908  
          EquiWidthHistogram                  13069.593868  
          GDYHistogram                        12783.414794  
          MaxDiffFrequencyHistogram            4230.731778  
TPC-DS    EqualDistinctCountHistogram         20084.507124  
          EquiHeightHistogram                 21770.434396  
          EquiWidth

### without_NULLs

In [8]:
size_before = len(df)
df = df.drop(df[df.real_output == 0].index)
print(f"Removed {len(df)-size_before} ({(size_before-len(df))/size_before:.2%}) rows because the 'real_output' cell was zero.")
df = df.drop(df[df.estimated_output == 0].index)
print(f"Removed {len(df)-size_before} ({(size_before-len(df))/size_before:.2%}) rows because the 'estimated_output' cell was zero.")

df_with_errors = generate_errors(df)

df_with_errors.groupby([#"operator_type",
                        "benchmark",
                        "histogram"]).agg(root_mean_squared_error=("mean_squared_error", lambda x: np.sqrt(np.mean(x))),
                                          mean_absolute_error=("absolute_error", np.mean),
                                          mean_relative_error=("relative_error", np.mean),
                                          mean_q_error=("q_error", np.mean),
                                          mean_pseudo_q_error=("pseudo_q_error", np.mean))

Removed 0 (0.00%) rows because the 'real_output' cell was zero.
Removed 0 (0.00%) rows because the 'estimated_output' cell was zero.


root_mean_squared_error  \
benchmark histogram                                              
JoinOrder EqualDistinctCountHistogram             4.940210e+05   
          EquiHeightHistogram                     5.262089e+05   
          EquiWidthHistogram                      5.270674e+05   
          GDYHistogram                            5.232776e+05   
          MaxDiffFrequencyHistogram               5.305747e+05   
TPC-DS    EqualDistinctCountHistogram             1.424409e+06   
          EquiHeightHistogram                     1.376252e+06   
          EquiWidthHistogram                      1.451672e+06   
          GDYHistogram                            1.476498e+06   
          MaxDiffFrequencyHistogram               1.316996e+06   
TPC-H     EqualDistinctCountHistogram             9.594396e+06   
          EquiHeightHistogram                     8.768265e+06   
          EquiWidthHistogram                      8.888339e+06   
          GDYHistogram                            8.589895e+06   
          MaxDiffFrequencyHistogram               9.774818e+06   

                                       mean_absolute_error  \
benchmark histogram                                          
JoinOrder EqualDistinctCountHistogram         1.374440e+05   
          EquiHeightHistogram                 1.281059e+05   
          EquiWidthHistogram                  1.304745e+05   
          GDYHistogram                        1.297047e+05   
          MaxDiffFrequencyHistogram           1.323861e+05   
TPC-DS    EqualDistinctCountHistogram         6.173912e+05   
          EquiHeightHistogram                 6.082235e+05   
          EquiWidthHistogram                  6.554053e+05   
          GDYHistogram                        6.198807e+05   
          MaxDiffFrequencyHistogram           5.635780e+05   
TPC-H     EqualDistinctCountHistogram         2.229261e+06   
          EquiHeightHistogram                 2.140811e+06   
          EquiWidthHistogram                  2.267699e+06   
          GDYHistogram                        1.974556e+06   
          MaxDiffFrequencyHistogram           2.470606e+06   

                                       mean_relative_error  mean_q_error  \
benchmark histogram                                                        
JoinOrder EqualDistinctCountHistogram           437.373708  2.445246e+06   
          EquiHeightHistogram                   657.882791  8.415922e+05   
          EquiWidthHistogram                    565.673177  2.689298e+05   
          GDYHistogram                          400.988069  5.501156e+05   
          MaxDiffFrequencyHistogram             647.162071  5.662291e+03   
TPC-DS    EqualDistinctCountHistogram            65.115598  4.090396e+09   
          EquiHeightHistogram                    54.352807  1.896268e+09   
          EquiWidthHistogram                     22.678550  6.732727e+08   
          GDYHistogram                           46.576783  3.654381e+09   
          MaxDiffFrequencyHistogram              69.237813  5.677187e+08   
TPC-H     EqualDistinctCountHistogram         33776.448426  3.379499e+04   
          EquiHeightHistogram                 31340.652350  3.135492e+04   
          EquiWidthHistogram                  33388.680264  3.339581e+04   
          GDYHistogram                        32473.407134  3.248194e+04   
          MaxDiffFrequencyHistogram           44844.617943  4.507321e+04   

                                       mean_pseudo_q_error  
benchmark histogram                                         
JoinOrder EqualDistinctCountHistogram         28204.870313  
          EquiHeightHistogram                  9935.452951  
          EquiWidthHistogram                  11409.248968  
          GDYHistogram                        11081.117593  
          MaxDiffFrequencyHistogram            2153.357017  
TPC-DS    EqualDistinctCountHistogram         16899.077654  
          EquiHeightHistogram                 15156.255048  
          EquiWidth

## compare_operators

In [15]:
grouped = df.groupby(["benchmark", "histogram", "query", "operator_type"]).size()

queries = df[["benchmark", "query"]].drop_duplicates()

# super inefficient, doesn't matter for now
op_counts = {}
for _, outer_benchmark, outer_query in queries.itertuples():
    for (_1, benchmark, histogram, query, operator_type, count) in grouped.reset_index().itertuples():
        if outer_benchmark == benchmark and outer_query == query:
            if (benchmark, query, operator_type) not in op_counts:
                op_counts[(benchmark, query, operator_type)] = count
                continue

            if op_counts[(benchmark, query, operator_type)] != count:
                print(f"WARNING: different value of {count} (previously {op_counts[(benchmark, query, operator_type)]}) for {benchmark}, query {query}, op {operator_type}")
